In [2]:
import geopandas as gpd
import pandas as pd
import os
import requests
from io import StringIO

In [3]:
bikeTheftData = pd.read_csv('bike_theft.csv')
bikeTheftData

,LSOA Code,LSOA Name,Borough,202210,202211,202212,202301,202302,202303,202304,...,202312,202401,202402,202403,202404,202405,202406,202407,202408,202409
0,E01000007,Barking and Dagenham 015A,E09000002,0,2,0,0,2,0,0,...,0,1,1,0,1,0,0,0,1,0
1,E01000009,Barking and Dagenham 016B,E09000002,1,0,0,1,0,0,0,...,1,1,0,0,0,0,1,0,0,0
2,E01000011,Barking and Dagenham 016C,E09000002,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,E01000012,Barking and Dagenham 015D,E09000002,0,0,0,1,0,0,2,...,0,0,0,1,0,0,1,0,0,0
4,E01000013,Barking and Dagenham 013A,E09000002,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,E01035718,Westminster 019G,E09000033,6,1,1,1,2,1,2,...,0,1,4,3,2,1,4,8,3,3
3884,E01035719,Westminster 021F,E09000033,0,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3885,E01035720,Westminster 021G,E09000033,1,0,2,0,0,0,1,...,0,1,0,0,1,1,0,0,1,2
3886,E01035721,Westminster 023H,E09000033,1,1,0,0,1,1,0,...,0,0,1,2,1,0,0,1,0,1


In [4]:
month_columns = bikeTheftData.columns[3:] 
bikeTheftData[month_columns] = bikeTheftData[month_columns].apply(pd.to_numeric, errors='coerce')
bikeTheftData['Total'] = bikeTheftData[month_columns].sum(axis=1)
NewBikeTheftData = bikeTheftData[['LSOA Code','LSOA Name','Borough','Total']]
##NewBikeTheftData.to_csv('bikeTheftDataTotal.csv', index=False)
NewBikeTheftData

,LSOA Code,LSOA Name,Borough,Total
0,E01000007,Barking and Dagenham 015A,E09000002,15
1,E01000009,Barking and Dagenham 016B,E09000002,5
2,E01000011,Barking and Dagenham 016C,E09000002,1
3,E01000012,Barking and Dagenham 015D,E09000002,5
4,E01000013,Barking and Dagenham 013A,E09000002,3
...,...,...,...,...
3883,E01035718,Westminster 019G,E09000033,69
3884,E01035719,Westminster 021F,E09000033,6
3885,E01035720,Westminster 021G,E09000033,14
3886,E01035721,Westminster 023H,E09000033,18


In [5]:
NewBikeTheftData.loc[:, 'Total'] = pd.to_numeric(NewBikeTheftData['Total'], errors='coerce')
totalCounts = NewBikeTheftData['Total'].value_counts().reset_index()
totalCounts.columns = ['Total', 'Times']  
totalCounts = totalCounts.sort_values(by='Total').reset_index(drop=True)
totalCounts.to_csv('totalCounts.csv',index=False)
totalCounts

,Total,Times
0,1,835
1,2,524
2,3,427
3,4,339
4,5,244
...,...,...
90,148,1
91,163,1
92,192,1
93,196,1


In [6]:
stats_df = pd.read_csv('bikeTheftDataTotal.csv')
all_geojson = gpd.GeoDataFrame()

for code in stats_df['LSOA Code'].unique():
    api_url = f'https://findthatpostcode.uk/areas/{code}.geojson'
    response = requests.get(api_url)
    
    if response.status_code == 200:
        temp_gdf = gpd.read_file(StringIO(response.text))
        all_geojson = pd.concat([all_geojson, temp_gdf], ignore_index=True)

merged_gdf = all_geojson.merge(stats_df, left_on='LSOA21CD', right_on='LSOA Code', how='left')
merged_gdf.to_file('lsoaDataBikeTheft.geojson', driver='GeoJSON')

print("GeoJSON file successfully exported: lsoaDataBikeTheft.geojson")

Skipping field alternative_names: unsupported OGR type: 5


{"features":[{"geometry":{"coordinates":[[[[0.07776794800003,51.54279273600008],[0.079675463000058,51.541342532000044],[0.080280203000029,51.541699824000034],[0.080700684000021,51.54111682300004],[0.08212934900007,51.54082170000004],[0.083151964000024,51.54110930100006],[0.083208782000042,51.54109030600006],[0.083049039000059,51.54068504900005],[0.082095272000061,51.539905254000075],[0.081361417000039,51.53930711500004],[0.080664973000069,51.53875235400005],[0.079467296000075,51.539326233000054],[0.07871401400007,51.53961397000006],[0.078354613000045,51.53947034200007],[0.07805268900006,51.53933821800007],[0.077537551000034,51.539743024000074],[0.07735169800003,51.53967755000008],[0.076986119000026,51.53953282200007],[0.077734126000053,51.53915908400006],[0.076541118000023,51.53863876500003],[0.075529503000041,51.538403189000064],[0.075246027000048,51.53950226300003],[0.074758788000054,51.53939942300008],[0.073640867000051,51.53887116400006],[0.073069463000024,51.538789535000035],[0.07

Skipping field alternative_names: unsupported OGR type: 5


{"features":[{"geometry":{"coordinates":[[[[0.082373410000059,51.54274511700004],[0.084212875000048,51.54182709300005],[0.084773583000072,51.54249525800003],[0.086445629000025,51.54179862700005],[0.086482384000021,51.54177737100008],[0.086148346000073,51.54154002900003],[0.086525940000058,51.541252963000034],[0.085096458000066,51.540358422000054],[0.086809593000055,51.53944577100003],[0.086217349000037,51.53901528800003],[0.085800805000019,51.53917990700006],[0.084945093000044,51.53852164300008],[0.085341186000051,51.53836255700003],[0.085146480000049,51.53815551400004],[0.084998746000053,51.53789744900007],[0.081585275000066,51.53912812600004],[0.081361417000039,51.53930711500004],[0.082095272000061,51.539905254000075],[0.083049039000059,51.54068504900005],[0.083208782000042,51.54109030600006],[0.083151964000024,51.54110930100006],[0.08212934900007,51.54082170000004],[0.080700684000021,51.54111682300004],[0.080280203000029,51.541699824000034],[0.080537461000063,51.54189986700004],[0.0

Skipping field alternative_names: unsupported OGR type: 5


...

Skipping field alternative_names: unsupported OGR type: 5


{"features":[{"geometry":{"coordinates":[[[[0.156894535000049,51.53025727800008],[0.159041865000063,51.529505974000074],[0.160268410000072,51.53023588900004],[0.162516900000071,51.52778432900004],[0.162914373000035,51.52801407700008],[0.162695500000041,51.52833370500008],[0.16337750100007,51.52829905300007],[0.163880401000029,51.52918130000006],[0.163393724000059,51.52995031000006],[0.164000462000047,51.530116546000045],[0.166012533000071,51.52880064300007],[0.164981169000043,51.52752889200008],[0.164557688000059,51.52717307200004],[0.162318077000066,51.52194071800005],[0.158972135000056,51.51358623500005],[0.158694297000068,51.51224261100003],[0.158080107000046,51.51200536500005],[0.151713406000056,51.51274850600004],[0.146565251000027,51.514099086000044],[0.14648513000003,51.514418456000044],[0.145847081000056,51.51432382400003],[0.14507496300007,51.51462454600005],[0.145767558000045,51.51448418500007],[0.145506535000038,51.51514095400006],[0.144264373000055,51.51549999300005],[0.143

Skipping field alternative_names: unsupported OGR type: 5


{"features":[{"geometry":{"coordinates":[[[[0.106713134000074,51.531302346000075],[0.106773573000055,51.530400764000035],[0.105194515000051,51.53062659900007],[0.105098167000051,51.52967170000005],[0.104587066000022,51.529647529000044],[0.104506814000047,51.529241873000046],[0.105053703000067,51.52915599100004],[0.105316029000051,51.528591824000046],[0.106685408000033,51.52846982800003],[0.106600036000032,51.52750008800007],[0.105919724000046,51.52669432500005],[0.104934043000071,51.52614061000003],[0.103827237000075,51.52618615500006],[0.102583544000026,51.527425264000044],[0.099284917000034,51.527124137000044],[0.099439527000072,51.52668875900008],[0.098779467000043,51.52630026600008],[0.098146129000042,51.52692469900006],[0.09729797600005,51.52675049800007],[0.096795741000051,51.52712125800008],[0.097128921000035,51.527388433000056],[0.09653632900006,51.52735776600008],[0.096410233000029,51.52768101000004],[0.097250883000072,51.527818522000075],[0.097605422000072,51.52828083700007],

KeyboardInterrupt: 